In [1]:
from src import util
import tensorflow as tf
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sensor_data_trainval, sequence_ids_trainval, labels_trainval = util.ingest_sensor_data('train.csv', 'train_labels.csv')
sensor_data_test, sequence_ids_test, _ = util.ingest_sensor_data('test.csv')

sensor_data_train, sequence_ids_train, labels_train, sensor_data_val, sequence_ids_val, labels_val = util.validation_split(training_fraction=0.7, sensor_data=sensor_data_trainval, sequence_ids=sequence_ids_trainval, labels=labels_trainval)

(25968, 13, 60) 25968
(12218, 13, 60) 12218


In [4]:
n_train=sensor_data_train.shape[0]
n_val=sensor_data_val.shape[0]
n_trainval=sensor_data_trainval.shape[0]
n_test=sensor_data_test.shape[0]
x_raw_train = util.preprocess_sensor_data(sensor_data_train.reshape((n_train,-1)))
x_raw_val = util.preprocess_sensor_data(sensor_data_val.reshape((n_val,-1)))
x_raw_trainval = util.preprocess_sensor_data(sensor_data_trainval.reshape((n_trainval,-1)))
x_raw_test = util.preprocess_sensor_data(sensor_data_test.reshape((n_test,-1)))



In [5]:
x_raw_train.shape

(18177, 780)

In [ ]:
tf.__version__


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])

#model.summary()


In [ ]:
model.fit(x_raw_train, np.array(labels_train), epochs=50)

This model does not seem to learn much, which was to be expected from such a stupid model.

In [ ]:
type(labels_train)

In [ ]:
model_multilayer = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='relu'),  
  tf.keras.layers.Dense(10, activation='relu'),    
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model_multilayer.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


In [ ]:
model_multilayer.build()
model_multilayer.summary()

In [ ]:
model_multilayer.fit(x_raw_train, np.array(labels_train), epochs=50)

Even simplistic feeding all sensor outputs to multilayer net works surprisingly well 

In [6]:
# does clipping and scaling the input data help?

#yes, it seems to help some

model_multilayer_clip = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation='relu'),  
  tf.keras.layers.Dropout(0.5),    
  tf.keras.layers.Dense(10, activation='relu'),    
  tf.keras.layers.Dropout(0.5),      
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

model_multilayer_clip.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(name='auc')])


model_multilayer_clip.fit(x_raw_trainval, np.array(labels_trainval), validation_split=0.3, epochs=50)

Epoch 1/50
569/569 [==============================] - 2s 2ms/step - loss: 0.6955 - accuracy: 0.5143 - auc: 0.5241 - val_loss: 0.6881 - val_accuracy: 0.5333 - val_auc: 0.5522
Epoch 2/50
569/569 [==============================] - 1s 2ms/step - loss: 0.6900 - accuracy: 0.5249 - auc: 0.5390 - val_loss: 0.6882 - val_accuracy: 0.5383 - val_auc: 0.5598
Epoch 3/50
569/569 [==============================] - 1s 2ms/step - loss: 0.6873 - accuracy: 0.5351 - auc: 0.5539 - val_loss: 0.6870 - val_accuracy: 0.5425 - val_auc: 0.5624
Epoch 4/50
569/569 [==============================] - 1s 2ms/step - loss: 0.6847 - accuracy: 0.5479 - auc: 0.5699 - val_loss: 0.6869 - val_accuracy: 0.5404 - val_auc: 0.5603
Epoch 5/50
569/569 [==============================] - 1s 2ms/step - loss: 0.6837 - accuracy: 0.5455 - auc: 0.5733 - val_loss: 0.6881 - val_accuracy: 0.5386 - val_auc: 0.5576
Epoch 6/50
569/569 [==============================] - 1s 2ms/step - loss: 0.6797 - accuracy: 0.5462 - auc: 0.5832 - val_loss: 0.68

In [7]:
pred = model_multilayer_clip.predict(util.preprocess_sensor_data(x_raw_test))

In [8]:
pred.shape

(12218, 1)

In [11]:
np.unique(pred)

array([0.48586679], dtype=float32)

In [ ]:
util.generate_submission(pred, sequence_ids_test, 'raw-clip')